# Run published pipeline using its REST endpoint

To invoke the run of the preceding pipeline, you need an Azure Active Directory authentication header token, as described in AzureCliAuthentication class.

In mlops-publish-pipeline notebook you built and published a pipeline of your experiment, now you in this notebook you would like to execute/invoke this published pipeline via REST endpoint

### Imports

In [ ]:
import os
import sys
import requests
import azureml.core
import warnings
warnings.filterwarnings('ignore')

from azureml.core import Experiment, Workspace
from azureml.widgets import RunDetails
from azureml.pipeline.core import PipelineEndpoint, PublishedPipeline
from azureml.pipeline.core import PublishedPipeline, PipelineRun
from azureml.core.authentication import InteractiveLoginAuthentication

sys.path.append(os.path.abspath("../utils"))
from workspace import get_workspace
from pipeline_parameter_builder import PipelineParameterBuilder

### Initialize Workspace

Now you're ready to connect to your workspace using the Azure ML SDK.

Note: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [ ]:
ws = Workspace.from_config("../notebooks-settings/config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### Pipeline Parameter Builder

In [ ]:
ppb = PipelineParameterBuilder("../utils/params_config/pipeline_parameters.json")
ppb.build()

### Create request pipeline parameters

In [ ]:
pipeline_parameters = {
            "ExperimentName": "heart-disease",
            "RunSource": "",
            "ParameterAssignments": ppb.get_raw_pipeline_parameters()
}

In [ ]:
pipeline_parameters

### Create REST request

To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. A real application would require a service principal with which to be authenticated, but to test this out, we'll use the authorization header from your current connection to your Azure workspace, which you can get using the following code:

In [ ]:
cli_auth = InteractiveLoginAuthentication()
aad_token = cli_auth.get_authentication_header()

pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name="<endpoint name that you put in mlops publish notebook>")

print("The published pipeline REST endpoint is {}".format(pipeline_endpoint))

Now we're ready to call the REST interface. The pipeline runs asynchronously, so we'll get an identifier back, which we can use to track the pipeline experiment as it runs:

In [ ]:
response = requests.post(pipeline_endpoint.endpoint,
                         headers=aad_token,
                         json=pipeline_parameters
                        )
pipeline_run_id = response.json()["Id"]

### Examine the REST execution run
Since we have the run ID, we can use the RunDetails widget. We can examine the run of the pipeline that we just invoked via the REST endpoint.

In [ ]:
pub_pipeline_run = PipelineRun(Experiment(ws, "heart-disease"), pipeline_run_id)
RunDetails(pub_pipeline_run).show()

#### Get additonal run details
If you wait until the pipeline_run is finished, you may be able to get additional details on the run. **Since this is a blocking call, the following code is commented out.**

In [ ]:
pub_pipeline_run.wait_for_completion()
for step_run in pub_pipeline_run.get_children():
    print("{}: {}".format(step_run.name, step_run.get_metrics()))